## Imports

In [44]:
from glob import glob
from openpyxl import load_workbook
from random import randint

## Configuration

In [45]:
#mode = "lines"
#mode = "phrases"
mode = "sentences"

In [46]:
if mode == "phrases":
    match_lines = False
    word_split_token = " "
    characters_to_replace_with_word_split_token = ",!?\n"

In [47]:
if mode == "sentences":
    match_lines = False
    word_split_token = "."
    characters_to_replace_with_word_split_token = "\n"

In [48]:
if mode == "lines":
    match_lines = True
    word_split_token = "."
    characters_to_replace_with_word_split_token = "\n"

### Other Config

In [49]:
min_phrase_length = 8

If you want to specify particular files, do so here. If no files are defined, then all matches to "books/\*.txt" will be used.

input_files = ["books/The Illustrated Complete Works - H. P. Lovecraft.txt"]

input_files = [
    "books/conan/The Hour of the Dragon - Robert E. Howard.txt",
]

input_files = glob("books/conan/*.txt")

In [50]:
input_files = glob("books/occult horror/*.txt")
output_file_name = "occult_horror_sentences.xlsx"

In [51]:
if not input_files:
    input_files = glob("books/*.txt")

output_file_name = "lovecraft.xlsx"

In [52]:
input_files

['books/occult horror\\Carmilla - Joseph Sheridan le Fanu.txt',
 'books/occult horror\\Dracula - Bram Stoker.txt',
 'books/occult horror\\Ghost Story.txt',
 'books/occult horror\\Psychic Self-Defense - Dion Fortune.txt',
 'books/occult horror\\The Night Season.txt',
 'books/occult horror\\The Secrets of Dr. Taverner - Dion Fortune.txt']

## Book Chopping

In [53]:
def num_words(token, a=2, b=7):
    if token == ".":
        return 1
    else:
        return randint(a, b) + randint(a, b)

In [54]:
def clean(phrase: str):
    phrase = phrase.replace("â€", ",")
    for char in characters_to_replace_with_word_split_token:
        phrase = phrase.replace(char, word_split_token)
    return phrase.strip()

In [55]:
def get_words(input_files:list, word_split_token:str):
    words = []
    for book_file in input_files:
        with open(book_file, encoding="utf-8") as book:
            words.extend(clean(book.read()).split(word_split_token))
    return words

In [56]:
def get_phrases_random(input_files, word_split_token):
    phrases = []
    words = get_words(input_files, word_split_token)
    while True:
        try:
            n = num_words(word_split_token)
            phrase = [words.pop(0) for _ in range(n)]
        except IndexError:
            phrase = words
            break
        finally:
            phrase = " ".join(phrase).strip()
            if len(phrase) > min_phrase_length:
                phrases.append(phrase)
    return phrases

In [57]:
def get_phrases_lines(input_files):
    lines = []
    for book_file in input_files:
        with open(book_file, encoding="utf-8") as book:
            for line in book:
                line = clean(line)
                if len(line) > min_phrase_length:
                    lines.append(line)
    return lines

In [58]:
def get_phrases(input_files, match_lines=False, word_split_token=" "):
    if match_lines:
        return get_phrases_lines(input_files)
    else:
        return get_phrases_random(input_files, word_split_token)

In [59]:
phrases = get_phrases(input_files, match_lines, word_split_token)

In [60]:
phrases[7639]

"I know them steps, 'avin' 'ad to carry the boxes up with three loafers what come round to earn a copper"

## File Writing

In [62]:
wb = load_workbook(filename="template.xlsx")
phrases_sheet = wb["phrases"]
for i, phrase in enumerate(phrases):
    phrases_sheet.append((i, phrase))
wb.save(output_file_name)